In [1]:
import sys
sys.path.append('..')
from dictionary_learning import AutoEncoder
from attribution import patching_effect
from nnsight import LanguageModel
from collections import namedtuple
import os
import torch as t

In [2]:
device = "cuda:0"
model_name = "EleutherAI/pythia-70m-deduped"
model = LanguageModel(model_name, dispatch=True, device_map=device)

In [3]:
Args = namedtuple('args', ['dict_id', 'd_model', 'dict_path', 'device'])
Ccfg = namedtuple('ccfg', ['dict_size'])
args = Args(d_model=512, dict_id=10, device=device, dict_path="/share/projects/dictionary_circuits/autoencoders/pythia-70m-deduped")
ccfg = Ccfg(dict_size=512*64)

In [4]:
embed = model.gpt_neox.embed_in
attns = [layer.attention for layer in model.gpt_neox.layers]
mlps = [layer.mlp for layer in model.gpt_neox.layers]
resids = [layer for layer in model.gpt_neox.layers]
submodules = [embed] + attns + mlps + resids
# save n_submodules to the config
# ccfg.n_submodules = 3 * model.config.num_hidden_layers + 1
# with open(os.path.join(ccfg.results_dir, "config.json"), "w") as f:
#     json.dump(ccfg.__dict__, f)

dictionaries = {}
if args.dict_id == 'id':
    from dictionary_learning.dictionary import IdentityDict
    dictionaries[embed] = IdentityDict(args.d_model)
    for i in range(len(model.gpt_neox.layers)):
        dictionaries[attns[i]] = IdentityDict(args.d_model)
        dictionaries[mlps[i]] = IdentityDict(args.d_model)
        dictionaries[resids[i]] = IdentityDict(args.d_model)
else:
    for i in range(len(model.gpt_neox.layers)):
        ae = AutoEncoder(args.d_model, ccfg.dict_size).to(args.device)
        ae.load_state_dict(t.load(os.path.join(args.dict_path, f'embed/{args.dict_id}_{ccfg.dict_size}/ae.pt')))
        dictionaries[embed] = ae

        ae = AutoEncoder(args.d_model, ccfg.dict_size).to(args.device)
        ae.load_state_dict(t.load(os.path.join(args.dict_path, f'attn_out_layer{i}/{args.dict_id}_{ccfg.dict_size}/ae.pt')))
        dictionaries[attns[i]] = ae

        ae = AutoEncoder(args.d_model, ccfg.dict_size).to(args.device)
        ae.load_state_dict(t.load(os.path.join(args.dict_path, f'mlp_out_layer{i}/{args.dict_id}_{ccfg.dict_size}/ae.pt')))
        dictionaries[mlps[i]] = ae

        ae = AutoEncoder(args.d_model, ccfg.dict_size).to(args.device)
        ae.load_state_dict(t.load(os.path.join(args.dict_path, f'resid_out_layer{i}/{args.dict_id}_{ccfg.dict_size}/ae.pt')))
        dictionaries[resids[i]] = ae

In [10]:
is_tuple = {}
with model.trace("_"):
    for submodule in submodules:
        is_tuple[submodule] = type(submodule.output.shape) == tuple
        print(submodule.output.shape, 'hi')

torch.Size([1, 1, 512]) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
torch.Size([1, 1, 512]) hi
torch.Size([1, 1, 512]) hi
torch.Size([1, 1, 512]) hi
torch.Size([1, 1, 512]) hi
torch.Size([1, 1, 512]) hi
torch.Size([1, 1, 512]) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1

In [6]:
contexts

NameError: name 'contexts' is not defined

In [7]:
contexts = "The dog"
answers = "is"
contexts = model.tokenizer(contexts, return_tensors="pt", padding=False, truncation=True, max_length=10).to(device)
answers = model.tokenizer(answers, return_tensors="pt", padding=False, truncation=True, max_length=10).to(device)
contexts = contexts.input_ids
answers = answers.input_ids


def metric_fn(model):
        return (
            -1 * t.gather(
                t.nn.functional.log_softmax(model.embed_out.output[:,-1,:], dim=-1), dim=-1, index=answers.view(-1, 1)
            ).squeeze(-1)
        )

effect_out = patching_effect(
        clean=contexts,
        patch=None,
        model=model,
        submodules=submodules,
        dictionaries=dictionaries,
        metric_fn=metric_fn,
        method="all-folded",
    )

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([1, 1, 512]) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
torch.Size([1, 1, 512]) hi
torch.Size([1, 1, 512]) hi
torch.Size([1, 1, 512]) hi
torch.Size([1, 1, 512]) hi
torch.Size([1, 1, 512]) hi
torch.Size([1, 1, 512]) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1, 64]), torch.Size([1, 8, 1, 64]))) hi
(torch.Size([1, 1, 512]), (torch.Size([1, 8, 1

In [8]:
type(submodules[0].output.shape) == tuple

AttributeError: 'Embedding' object has no attribute 'output'

IndexError: list index out of range

In [20]:
effect_out

EffectOut(effects={Embedding(50304, 512): SparseAct(act=tensor([[[0., -0., -0.,  ..., -0., -0., -0.],
         [0., -0., 0.,  ..., 0., 0., 0.]]], device='cuda:0'), resc=tensor([[[ 2.4042e-04],
         [-8.6635e-05]]], device='cuda:0')), GPTNeoXAttention(
  (rotary_emb): GPTNeoXRotaryEmbedding()
  (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
  (dense): Linear(in_features=512, out_features=512, bias=True)
  (attention_dropout): Dropout(p=0.0, inplace=False)
): SparseAct(act=tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., -0., 0.,  ..., 0., 0., 0.]]], device='cuda:0'), resc=tensor([[[-0.0026],
         [-0.1496]]], device='cuda:0')), GPTNeoXAttention(
  (rotary_emb): GPTNeoXRotaryEmbedding()
  (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
  (dense): Linear(in_features=512, out_features=512, bias=True)
  (attention_dropout): Dropout(p=0.0, inplace=False)
): SparseAct(act=tensor([[[0., -0., 0.,  ..., 0., -0., 0.],
         [-0.,